In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

## Задание 1

Открой файл [distances.ipynb](src/notebooks/distances.ipynb). 
* Объедини общие данные о фильмах [tmdb_5000_movies](https://files.sberdisk.ru/s/te4QbzdxKgsFQXA) и каст фильмов 
[tmdb_5000_credits](https://files.sberdisk.ru/s/H9oRuXQt5mFz3T9). 
* Оставь в датасете только фильмы, которые вышли в "релиз".
* Убери фильмы с пропусками в колонках ['overview', 'genres', 'keywords']
* Выведи количество фильмов, оставшихся в выборке 

In [2]:
movies_df = pd.read_csv("../../datasets/tmdb_5000_movies.csv")
credits_df = pd.read_csv("../../datasets/tmdb_5000_credits.csv")

# Объединение данных о фильмах и касте
merged_df = pd.merge(movies_df, credits_df, left_on='id', right_on='movie_id')
merged_df.rename(columns={'title_x': 'title'}, inplace=True)

# Оставляем фильмы, которые вышли в релиз
movie_df = merged_df[merged_df["status"] == "Released"]

# Удаление фильмов с пропусками в колонках 'overview', 'genres', 'keywords'
movie_df = movie_df.dropna(subset=['overview', 'genres', 'keywords'])

# Вывод количества оставшихся фильмов
print("Количество фильмов, оставшихся в выборке:", movie_df.shape[0])

Количество фильмов, оставшихся в выборке: 4792


## Задание 2

Реализуем алгоритм рекомендации на основе описания фильма (`overview`) и ключевых слов к фильму (`keywords`). 
Объедини тексты этих колонок и проведи предобработку:
* Замени NaN в описании фильма на пустой символ `''`
* Удали все английские стоп-слова (используй параметр `stop_words` в `TfidfVectorizer`)
* Рассчитай матрицу [Tf-Idf](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) для описания фильмов.

Выведи размер получившейся матрицы
> Параметр `max_features` в `TfidfVectorizer` должен быть равен 10000

In [3]:
# Замена NaN в описании фильма и ключевых словах на пустой символ
movie_df['overview'].fillna('', inplace=True)
movie_df['keywords'].fillna('', inplace=True)

# Объединение текстов описания и ключевых слов
text_data = movie_df['overview'] + ' ' + movie_df['keywords']

# Рассчет матрицы Tf-Idf для описания фильмов
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)
tfidf_matrix = tfidf.fit_transform(text_data)

print("Размер матрицы Tf-Idf:", tfidf_matrix.shape)

Размер матрицы Tf-Idf: (4792, 10000)


## Задание 3

Рассчитай [cosine similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) 
между фильмами. Составь из этой матрицы `pd.DataFrame`. Для дальнейшего удобства, 
колонки и индексы таблицы назови согласно`id` фильма. \
Сохрани получившийся `DataFrame` c расстояниями в папку [assets](src/assets) с названием `distance.csv`.
А сам объединенный датасет с фильмами сохрани в папку [assets](src/assets) с названием `movies.csv`.

> Получившиеся файлы `distance.csv` и `movies.csv` пушить в GitLab не нужно!

In [4]:
# Рассчет cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix)

# Создание DataFrame с расстояниями
distance_df = pd.DataFrame(cosine_sim, index=movie_df['id'], columns=movie_df['id'])

# Сохранение DataFrame с расстояниями в distance.csv
distance_df.to_csv("../assets/distance.csv")

# Установка индекса и колонок согласно id фильма
distance_df.index.name = 'id'
distance_df.columns.name = 'id'

# Сохранение объединенного датасета с фильмами в movies.csv
movie_df.to_csv("../assets/movies.csv", index=False)